## Wikidata linked data

Collect wikidata ids from lists in folder `inferences/wikidata-referencees`

In [2]:
import glob
import pandas as pd

# get list of all csv files in directory `../inferences/wikidata-references/`
csv_files = glob.glob("../inferences/wikidata-references/*.csv")
wikidata_entities = set()
for file in csv_files:
    print(file)
    df = pd.read_csv(file)
    # Assuming the wikidata id column is named 'wikidata_id'
    if 'wikidata_id' in df.columns:
        wikidata_entities.update(df['wikidata_id'].dropna().astype(str))
    print(len(df))
wikidata_entities = list(set(wikidata_entities))
print(len(wikidata_entities))


../inferences/wikidata-references/residences-1701.csv
144
../inferences/wikidata-references/residences-1644-1701.csv
311
../inferences/wikidata-references/locations_names_wikidata.csv
923
../inferences/wikidata-references/residences-1644.csv
167
1055


## Fetch wikidata data

In [3]:
%pip install pywikibot

.bash_profile RUN!
Note: you may need to restart the kernel to use updated packages.


Get all the wikidata ids in the database


In [10]:
from dehergne_util import locations_wikidata_info_file

# preload dataframe entities_df from xls file "../inferences/locations_wikidata_info.xlsx"
entities_cache = pd.read_excel(locations_wikidata_info_file, index_col=0)


In [9]:
entities_cache.loc['Q1011103',]


chinese_label                                                      富阳区
english_label                                          Fuyang District
portuguese_label                                                Fuyang
english_description               district of Zhejiang Province, China
chinese_description                                       中国浙江省杭州市的市辖区
portuguese_description                                             NaN
coordinates                                      (30.04998, 119.93697)
latitude                                                      30.04998
longitude                                                    119.93697
administrative_entity_id                                         Q4970
administrative_entity_label_en                                Hangzhou
administrative_entity_label_zh                                     杭州市
country_id                                                        Q148
country_label                               People's Republic of China
label 

In [12]:
entities_cache.sample(10)

,chinese_label,english_label,portuguese_label,english_description,chinese_description,portuguese_description,coordinates,latitude,longitude,administrative_entity_id,administrative_entity_label_en,administrative_entity_label_zh,country_id,country_label,label
wikidata_id,,,,,,,,,,,,,,,
Q857618,莎车县,Yarkant County,Yarkand,"county in Xinjiang, People's Republic of China",中国新疆维吾尔自治区喀什地区下辖县,NaN,"(38.41446, 77.24964)",38.414460,77.249640,Q739890,Kashgar Prefecture,喀什地区,Q148,People's Republic of China,Yarkant County / 莎车县
Q1064185,王府井天主堂,"St. Joseph's Church, Beijing",NaN,Beijing,NaN,NaN,"(39.915833333333, 116.40583333333)",39.915833,116.405833,Q956,Beijing,北京市,Q148,People's Republic of China,"St. Joseph's Church, Beijing / 王府井天主堂"
Q185221,马六甲,Malacca,Malaca,Malaysian state,马来西亚州属,NaN,"(2.2, 102.251)",2.200000,102.251000,Q833,Malaysia,馬來西亞,Q833,Malaysia,Malacca / 马六甲
Q1198385,沙县区,Shaxian District,NaN,"district in Sanming, Fujian, China",中国福建省三明市的市辖区,NaN,"(26.40278, 117.78529)",26.402780,117.785290,Q68614,Sanming,三明市,Q148,People's Republic of China,Shaxian District / 沙县区
Q720252,班多拉,Bandora,NaN,human settlement,NaN,NaN,"(15.4, 73.97)",15.400000,73.970000,Q108234,North Goa district,北果阿縣,Q668,India,Bandora / 班多拉
Q14960,布尔诺,Brno,Brno,"statutory city in South Moravian Region, the C...",捷克城市,NaN,"(49.195277777778, 16.608333333333)",49.195278,16.608333,Q850531,Brno-City District,布爾諾城縣,Q213,Czech Republic,Brno / 布尔诺
Q208471,開封市,Kaifeng,Kaifeng,"prefecture-level city in Henan, China, a forme...",中國河南省的地級市,NaN,"(34.795, 114.345)",34.795000,114.345000,Q43684,Henan,河南省,Q148,People's Republic of China,Kaifeng / 開封市
Q19944,切瓦,Ceva,Ceva,Italian comune,意大利的一个市镇,comuna italiana,"(44.383333333333, 8.0333333333333)",44.383333,8.033333,Q15091,Province of Cuneo,庫內奧省,Q38,Italy,Ceva / 切瓦
Q1362530,介休市,Jiexiu,Jiexiu,"county-level city in Shanxi, China",中国山西省城市,NaN,"(37.02821, 111.91028)",37.028210,111.910280,Q73045,Jinzhong,晋中市,Q148,People's Republic of China,Jiexiu / 介休市


In [13]:
import time
import pywikibot
import pandas as pd
from dehergne_util import locations_wikidata_info_file

site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()

entities_data = []
entities_from_cache = []
problems = []  # to store any problems encountered during the fetching process
for wikidata_id in wikidata_entities:
    if wikidata_id == 'Q11146687':
        pass
    # check if the wikidata_id is already in the cache
    if wikidata_id in entities_cache.index:
        entities_from_cache.append(wikidata_id)
        # print(f"{wikidata_id} already in cache, skipping...")
        continue
    entity_data = {}
    # Fetch the item from Wikidata
    entity_data['wikidata_id'] = wikidata_id
    print(f"Fetching {wikidata_id} from Wikidata...")
    # Use the repository to get the item
    try:
        item = pywikibot.ItemPage(repo, wikidata_id)
    except Exception as e:
        problems.append({'wikidata_id': wikidata_id, 'exception': str(e)})
        continue
    item_dict = item.get()
    entity_data['chinese_label'] = item_dict['labels'].get('zh', None)
    entity_data['english_label'] = item_dict['labels'].get('en', None)
    entity_data['portuguese_label'] = item_dict['labels'].get('pt', None)
    entity_data['english_description'] = item_dict['descriptions'].get('en', None)
    entity_data['chinese_description'] = item_dict['descriptions'].get('zh', None)
    entity_data['portuguese_description'] = item_dict['descriptions'].get('pt', None)
    # get the coordinates if they exist
    coord = None
    if 'P625' in item_dict['claims']:
        coord_claim = item_dict['claims']['P625'][0].getTarget()
        coord = (coord_claim.lat, coord_claim.lon)
    entity_data['coordinates'] = coord
    # store lat and lon in separate keys
    entity_data['latitude'] = coord[0] if coord else None
    entity_data['longitude'] = coord[1] if coord else None
    # get the administrative territorial entity if it exists (P131)
    administrative_entity = None
    administrative_entity_label_en = None
    administrative_entity_label_zh = None
    administrative_entity_id = None
    administrative_entity_claims = item_dict['claims'].get('P131', [])
    if administrative_entity_claims:
        administrative_entity = administrative_entity_claims[0].getTarget()
        administrative_entity_id = administrative_entity.id
        administrative_entity_label_en = administrative_entity.get()['labels'].get('en', None)
        administrative_entity_label_zh = administrative_entity.get()['labels'].get('zh', None)
    # store administrative information in entity_data
    entity_data['administrative_entity_id'] = administrative_entity_id
    entity_data['administrative_entity_label_en'] = administrative_entity_label_en
    entity_data['administrative_entity_label_zh'] = administrative_entity_label_zh
    # get the country
    country = None
    country_label = None
    country_id = None
    country_claims = item_dict['claims'].get('P17', [])
    if country_claims:
        country = country_claims[0].getTarget()
        country_id = country.id
        country_label = country.get()['labels'].get('en', None)
    entity_data['country_id'] = country_id
    entity_data['country_label'] = country_label
    # append the entity data to the list
    entities_data.append(entity_data)

    print(f"Fetched {wikidata_id} with labels: {entity_data['english_label']}, {entity_data['portuguese_label']}, {entity_data['chinese_label']}, {entity_data['administrative_entity_label_en']}, {entity_data['administrative_entity_label_zh']}, {entity_data['country_label']}")
    time.sleep(0.5)  # to avoid hitting the API too hard

# Convert the list of dictionaries to a DataFrame
entities_df = pd.DataFrame(entities_data)
if len(entities_df) == 0:
    print("No new entities found.")
    entities_df = entities_cache  # use the cached data if no new entities found
else:
    print(f"Fetched {len(entities_df)} new entities from Wikidata.")
    entities_df.set_index('wikidata_id', inplace=True)

# get the rows entities_cache in entities_from_cache list of ids
# this removes from cache entities not in the current run
entities_cache = entities_cache[entities_cache.index.isin(entities_from_cache)]
# append the new data to the existing cache
entities_df = pd.concat([entities_cache, entities_df], axis=0)



ConnectionError: HTTPSConnectionPool(host='www.wikidata.org', port=443): Max retries exceeded with url: /w/api.php?action=query&meta=userinfo&uiprop=blockinfo%7Cgroups%7Chasmsg%7Cratelimits%7Crights&formatversion=2&maxlag=5&format=json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1693c67d0>: Failed to resolve 'www.wikidata.org' ([Errno 8] nodename nor servname provided, or not known)"))

In [14]:
# print the problems encountered one per line
for problem in problems:
    print(f"Problem with {problem['wikidata_id']}: {problem['exception']}")

NameError: name 'problems' is not defined

In [ ]:
# show duplicates in the entities_df use index
duplicates = entities_df.index[entities_df.index.duplicated()].unique()
duplicates

Index([], dtype='object', name='wikidata_id')

In [ ]:
# remove duplicates from entities_df
print(f"Removing {len(duplicates)} duplicates from entities_df, before removal: {len(entities_df)} rows.")
entities_df = entities_df[~entities_df.index.duplicated(keep='first')]
print(f"After removal: {len(entities_df)} rows.")

Removing 0 duplicates from entities_df, before removal: 1075 rows.
After removal: 1075 rows.


In [ ]:
entities_df.info()
entities_df.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 1075 entries, Q90 to Q950944
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   chinese_label                   1016 non-null   object 
 1   english_label                   1070 non-null   object 
 2   portuguese_label                889 non-null    object 
 3   english_description             1057 non-null   object 
 4   chinese_description             770 non-null    object 
 5   portuguese_description          619 non-null    object 
 6   coordinates                     1036 non-null   object 
 7   latitude                        1036 non-null   float64
 8   longitude                       1036 non-null   float64
 9   administrative_entity_id        971 non-null    object 
 10  administrative_entity_label_en  967 non-null    object 
 11  administrative_entity_label_zh  856 non-null    object 
 12  country_id                      10

,chinese_label,english_label,portuguese_label,english_description,chinese_description,portuguese_description,coordinates,latitude,longitude,administrative_entity_id,administrative_entity_label_en,administrative_entity_label_zh,country_id,country_label,label
wikidata_id,,,,,,,,,,,,,,,
Q90,巴黎,Paris,Paris,capital city and largest city of France,法國首都及最大都市,capital e maior cidade da França,"(48.85666666666667, 2.352222222222222)",48.856667,2.352222,Q16665915,Grand Paris,大巴黎都会区,Q142,France,Paris / 巴黎
Q41185,卡昂,Caen,Caen,"commune in Calvados, France",法国卡尔瓦多斯省市镇与省会,Comuna francesa,"(49.18138888888889, -0.3636111111111111)",49.181389,-0.363611,Q700794,arrondissement of Caen,卡昂区,Q142,France,Caen / 卡昂
Q869,泰國,Thailand,Tailândia,country in Southeast Asia,東南亞國家,país da Ásia,"(14, 101)",14.000000,101.000000,NaN,NaN,NaN,Q869,Thailand,Thailand / 泰國
Q12193,布雷斯特,Brest,Brest,"port city in the Finistère department, Brittan...",法国菲尼斯泰尔省市镇与副省会,comuna francesa,"(48.39, -4.4869444444444)",48.390000,-4.486944,Q700737,arrondissement of Brest,布雷斯特区,Q142,France,Brest / 布雷斯特
Q639421,本地治里市,Pondicherry,NaN,"city in the Union Territory of Puducherry, India",NaN,NaN,"(11.93, 79.83)",11.930000,79.830000,Q66743,Puducherry,本地治里,Q668,India,Pondicherry / 本地治里市
Q1352,金奈,Chennai,Chennai,"city and state capital of Tamil Nadu, India","印度城市,泰米爾納德邦首府","cidade e capital do estado de Tamil Nadu, Índia","(13.0825, 80.275)",13.082500,80.275000,Q15116,Chennai district,金奈縣,Q668,India,Chennai / 金奈
Q622778,東京,Tonkin,Tonkin,"northern part of Vietnam, to the west of the G...",印度支那的历史地名，位于今日越南北部,NaN,"(21, 106)",21.000000,106.000000,NaN,NaN,NaN,Q881,Vietnam,Tonkin / 東京
Q16572,广州市,Guangzhou,Cantão,"capital city of Guangdong Province, China",广东省的省会,capital da província chinesa de Cantão,"(23.13, 113.26)",23.130000,113.260000,Q15175,Guangdong,广东省,Q148,People's Republic of China,Guangzhou / 广州市
Q15175,广东省,Guangdong,Cantão,province of China,中华人民共和国省份,província da China,"(23.4, 113.5)",23.400000,113.500000,Q148,People's Republic of China,中华人民共和国,Q148,People's Republic of China,Guangdong / 广东省


In [ ]:
entities_df.loc['Q1197421',]

chinese_label                                                     武进区
english_label                                          Wujin District
portuguese_label                                                  NaN
english_description               district of Jiangsu Province, China
chinese_description                                        江蘇省常州市的市辖区
portuguese_description                                            NaN
coordinates                                      (31.70204, 119.9377)
latitude                                                     31.70204
longitude                                                    119.9377
administrative_entity_id                                       Q57970
administrative_entity_label_en                              Changzhou
administrative_entity_label_zh                                    常州市
country_id                                                       Q148
country_label                              People's Republic of China
label               

In [ ]:
from dehergne_util import locations_wikidata_info_file
entities_df.to_excel(locations_wikidata_info_file, index=True)
print(f"Saved {len(entities_df)} entities to {locations_wikidata_info_file}")

Saved 1075 entities to ../inferences/wikidata-references/locations_wikidata_info.xlsx


Entries with no coordinates

In [ ]:
%pip install plotly nbformat

.bash_profile RUN!
Note: you may need to restart the kernel to use updated packages.


In [8]:
import plotly.express as px
import plotly.io as pio

# Create a 'label' column by concatenating 'english_label' and 'chinese_label'
# Handle potential None values by replacing them with empty strings
entities_df['label'] = entities_df['english_label'].fillna('') + ' / ' + entities_df['chinese_label'].fillna('')

# Drop rows where latitude or longitude is NaN before plotting
entities_df_map = entities_df.dropna(subset=['latitude', 'longitude'])

fig = px.scatter_map(
    entities_df_map,
    lat="latitude",
    lon="longitude",
    hover_name="label",  # Use the new 'label' column for hover text
    text="label",        # Show the label on the map
    zoom=3,

)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(autosize=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig.update_traces(textfont=dict(size=6))
fig.show(config={"responsive": True})

# Export the figure to an HTML file
pio.write_html(fig,
                config={"responsive": True},
                include_plotlyjs=True,
                file='../inferences/map_all_locations.html',
                auto_open=True)

NameError: name 'entities_df' is not defined